# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66
1,All Time Low,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",510,A Party Song (The Walk of Shame),51
2,Nik & Jay,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",379,Pop-Pop!,9
3,Quad City DJ's,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",506,C'mon N' Ride It (The Train) (LP Version),49
4,Hoobastank,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",492,Born To Lead,94


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [15]:
def create_table(data_dict, primary_keys, table_name):
    """
    Create a table in Cassandra.
    The query is static and already optimized for this usecase.
    This function has no Return value.
    
    It's just a helper function and will print out the query by executing
    
    Properties:
        * `data_dict`: Its a dictionary for table value datatype pairs in form {"value_name": "datatype"}
        * `primary_keys`: List Strings of Column Names which should be Primary Keys
        * `table_name`: Name (Type: STRING) of database Table where dataframe needs to be inserted.
    """

    values = [f"\t{key} {datatype}," for key, datatype in data_dict.items()]
    values_string = "\n".join(values)
    primary_key_string = ", ".join(primary_keys)
    query = f"CREATE TABLE IF NOT EXISTS {table_name} (\n{values_string}\n\tPRIMARY KEY ({primary_key_string})\n)"
    print(query)
    session.execute(query)
    """
    session.execute(
        "CREATE TABLE IF NOT EXISTS songplayinfo ({}), PRIMARY KEY ({}))".format(
            ", ".join(values),
            ", ".join(primary_keys)
        )
    )
    """

In [10]:
def insert_data_into_table(df, table_name):
    """
    Insert a full Dataframe in a table.
    The query is static and already optimized for this usecase.
    This function has no Return value.
    
    It's just a helper function.
    
    Properties:
        * `df`: Pandas Dataframe of Dataset
        * `table_name`: Name (Type: STRING) of database Table where dataframe needs to be inserted.
    """
    query = f"""
        INSERT INTO {table_name} (
            artist,
            firstName,
            gender,
            itemInSession,
            lastName,
            length,
            level,
            location,
            sessionId,
            song,
            userId
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    """
    for i, row in df.iterrows():
        session.execute(
            query, (
                row['artist'], row['firstName'], row['gender'],
                int(row['itemInSession']), row['lastName'], row['length'],
                row['level'], row['location'], int(row['sessionId']),
                row['song'], int(row['userId'])
            )
        )

In [11]:
data_dictionary = {
    'artist': 'text',
    'firstName': 'text',
    'gender': 'varchar',
    'itemInSession': 'int',
    'lastName': 'text',
    'length': 'float',
    'level': 'text',
    'location': 'text',
    'sessionId': 'int',
    'song': 'text',
    'userId': 'int'
}

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1
> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4"

In [17]:
table_name_1 = 'songplayinfo_q1'

In [18]:
primary_keys = ['sessionId', 'itemInSession']

In [19]:
create_table(data_dictionary, primary_keys, table_name_1)

CREATE TABLE IF NOT EXISTS songplayinfo_q1 (
	artist text,
	firstName text,
	gender varchar,
	itemInSession int,
	lastName text,
	length float,
	level text,
	location text,
	sessionId int,
	song text,
	userId int,
	PRIMARY KEY (sessionId, itemInSession)
)


In [20]:
insert_data_into_table(df, table_name_1)

In [21]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = f"""
    SELECT artist, song, length FROM {table_name_1} WHERE sessionId=338 AND itemInSession=4;
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


#### Query 2
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [22]:
table_name_2 = 'songplayinfo_q2'

In [23]:
primary_keys = ['(userId, sessionId)', 'itemInSession']

In [24]:
create_table(data_dictionary, primary_keys, table_name_2)

CREATE TABLE IF NOT EXISTS songplayinfo_q2 (
	artist text,
	firstName text,
	gender varchar,
	itemInSession int,
	lastName text,
	length float,
	level text,
	location text,
	sessionId int,
	song text,
	userId int,
	PRIMARY KEY ((userId, sessionId), itemInSession)
)


In [25]:
insert_data_into_table(df, table_name_2)

In [26]:
# User ID is not specified in this query.
# But in one session can be just one user.
# Even by a logout a session should get destroyed
try:
    rows = session.execute(f"""
        SELECT artist, song, length FROM {table_name_2} WHERE userId=10 AND sessionId=182 ORDER BY itemInSession;
    """)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", length=333.7660827636719)
Row(artist='Three Drives', song='Greece 2000', length=411.6370849609375)
Row(artist='Sebastien Tellier', song='Kilometer', length=377.73016357421875)
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', length=181.2109832763672)


#### Query 3
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [27]:
table_name_3 = 'songplayinfo_q3'

In [28]:
primary_keys = ['song', 'firstName', 'lastName', 'userId']

In [29]:
create_table(data_dictionary, primary_keys, table_name_3)

CREATE TABLE IF NOT EXISTS songplayinfo_q3 (
	artist text,
	firstName text,
	gender varchar,
	itemInSession int,
	lastName text,
	length float,
	level text,
	location text,
	sessionId int,
	song text,
	userId int,
	PRIMARY KEY (song, firstName, lastName, userId)
)


In [30]:
insert_data_into_table(df, table_name_3)

In [31]:
try:
    rows = session.execute(f"""
        SELECT firstName, lastName FROM {table_name_3} WHERE song='All Hands Against His Own';
    """)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [16]:
session.execute(f"DROP TABLE {table_name_1};")

In [87]:
session.execute(f"DROP TABLE {table_name_2};")

In [104]:
session.execute(f"DROP TABLE {table_name_3};")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()